# Fuel pin: 1A

## Import modules

In [1]:
import sys
import os

import openmc
import openmc.mgxs as mgxs

import numpy as np
import matplotlib.pyplot as plt
import copy as copy

import pandas as pd
from IPython.display import Image
import glob

## Add location of OpenMC XS executable, setup ENDF xs path

In [2]:
import os

# Add path to OpenMC binary
# os.environ['PATH'] += r':/path/to/openmc/bin'
os.environ['PATH'] += r':/Users/jean.ragusa/repo/openmc/local/bin'

# Add location of OpenMC xs data
%env OPENMC_CROSS_SECTIONS=/Users/jean.ragusa/repo/endfb-viii.0-hdf5/cross_sections.xml

env: OPENMC_CROSS_SECTIONS=/Users/jean.ragusa/repo/endfb-viii.0-hdf5/cross_sections.xml


## Start model

In [3]:
model=openmc.Model()

### Define Materials

In [4]:
uo2 = openmc.Material(name='uo2')

uo2.add_nuclide('U234', 6.11864E-06, 'ao')
uo2.add_nuclide('U235', 7.18132E-04, 'ao')
uo2.add_nuclide('U236', 3.29861E-06, 'ao')
uo2.add_nuclide('U238', 2.21546E-02, 'ao')
uo2.add_nuclide('O16', 4.57642E-02, 'ao')

uo2.set_density('g/cm3', 10.257 )

uo2.temperature = 565

In [5]:
zirconium = openmc.Material(name="zirconium")

zirconium.add_nuclide('Zr90', 2.18865E-02, 'ao')
zirconium.add_nuclide('Zr91', 4.77292E-03, 'ao')
zirconium.add_nuclide('Zr92', 7.29551E-03, 'ao')
zirconium.add_nuclide('Zr94', 7.39335E-03, 'ao')
zirconium.add_nuclide('Zr96', 1.19110E-03, 'ao')
zirconium.add_nuclide('Sn112',4.68066E-06, 'ao')
zirconium.add_nuclide('Sn114', 3.18478E-06, 'ao')
zirconium.add_nuclide('Sn115', 1.64064E-06, 'ao')
zirconium.add_nuclide('Sn116', 7.01616E-05, 'ao')
zirconium.add_nuclide('Sn117', 3.70592E-05, 'ao')
zirconium.add_nuclide('Sn118', 1.16872E-04, 'ao')
zirconium.add_nuclide('Sn119', 4.14504E-05, 'ao')
zirconium.add_nuclide('Sn120', 1.57212E-04, 'ao')
zirconium.add_nuclide('Sn122', 2.23417E-05, 'ao')
zirconium.add_nuclide('Sn124', 2.79392E-05, 'ao')
zirconium.add_nuclide('Fe54', 8.68307E-06, 'ao')
zirconium.add_nuclide('Fe56', 1.36306E-04, 'ao')
zirconium.add_nuclide('Fe57', 3.14789E-06, 'ao')
zirconium.add_nuclide('Fe58', 4.18926E-07, 'ao')
zirconium.add_nuclide('Cr50', 3.30121E-06, 'ao')
zirconium.add_nuclide('Cr52', 6.36606E-05, 'ao')
zirconium.add_nuclide('Cr53', 7.21860E-06, 'ao')
zirconium.add_nuclide('Cr54', 1.79686E-06, 'ao')
zirconium.add_nuclide('Hf174', 3.54138E-09, 'ao')
zirconium.add_nuclide('Hf176', 1.16423E-07, 'ao')
zirconium.add_nuclide('Hf177', 4.11686E-07, 'ao')
zirconium.add_nuclide('Hf178', 6.03806E-07, 'ao')
zirconium.add_nuclide('Hf179', 3.01460E-07, 'ao')
zirconium.add_nuclide('Hf180', 7.76449E-07, 'ao')

zirconium.set_density('g/cm3',  6.56)

zirconium.temperature = 565

In [6]:
water = openmc.Material(name="water")

water.add_nuclide('H1',4.96224E-02, 'ao')
water.add_nuclide('O16', 2.48112E-02, 'ao')
water.add_nuclide('B10', 1.07070E-05, 'ao')
water.add_nuclide('B11', 4.30971E-05, 'ao')

water.add_s_alpha_beta('c_H_in_H2O')

water.set_density('g/cm3', 0.743)

water.temperature=565

In [7]:
helium = openmc.Material(name="helium")

helium.add_nuclide('He4',1, 'ao')

helium.set_density('g/cm3', 0.178E-03 )

helium.temperature=565

In [8]:
model.materials = openmc.Materials([uo2, zirconium, water,helium]) 

print(model.materials)

[Material
	ID             =	1
	Name           =	uo2
	Temperature    =	565
	Density        =	10.257 [g/cm3]
	Volume         =	None [cm^3]
	Depletable     =	True
	S(a,b) Tables  
	Nuclides       
	U234           =	6.11864e-06  [ao]
	U235           =	0.000718132  [ao]
	U236           =	3.29861e-06  [ao]
	U238           =	0.0221546    [ao]
	O16            =	0.0457642    [ao]
, Material
	ID             =	2
	Name           =	zirconium
	Temperature    =	565
	Density        =	6.56 [g/cm3]
	Volume         =	None [cm^3]
	Depletable     =	False
	S(a,b) Tables  
	Nuclides       
	Zr90           =	0.0218865    [ao]
	Zr91           =	0.00477292   [ao]
	Zr92           =	0.00729551   [ao]
	Zr94           =	0.00739335   [ao]
	Zr96           =	0.0011911    [ao]
	Sn112          =	4.68066e-06  [ao]
	Sn114          =	3.18478e-06  [ao]
	Sn115          =	1.64064e-06  [ao]
	Sn116          =	7.01616e-05  [ao]
	Sn117          =	3.70592e-05  [ao]
	Sn118          =	0.000116872  [ao]
	Sn119          =	4.14504e-05 

### Create fuel pin: fuel, clad, water (no gap)

In [9]:
pitch = 1.26

fuel_outer_radius = openmc.ZCylinder(r=0.4096)
clad_inner_radius = openmc.ZCylinder(r=0.418)
clad_outer_radius = openmc.ZCylinder(r=0.475)

rod_height=1
left = openmc.XPlane(-pitch/2, boundary_type='reflective')
right = openmc.XPlane(pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
top = openmc.YPlane(pitch/2, boundary_type='reflective')

fuel_region = -fuel_outer_radius

gap_region = +fuel_outer_radius & -clad_inner_radius

clad_region = +clad_inner_radius & -clad_outer_radius

water_region = +left & -right & +bottom & -top & +clad_outer_radius

fuel = openmc.Cell(name='fuel')
fuel.fill = uo2
fuel.region = fuel_region

gap = openmc.Cell(name='gap')
gap.region = gap_region
gap.fill = helium 

clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region 

root_universe = openmc.Universe(cells=(fuel ,gap, clad, moderator))

model.geometry = openmc.Geometry(root_universe)

###  Settings

In [16]:
settings = openmc.Settings()

bounds = [-pitch/2, -pitch/2, -rod_height/2, pitch/2, +pitch/2, rod_height/2]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)

source = openmc.IndependentSource(space=uniform_dist)

settings.source = source

settings.batches = 50
settings.inactive = 20
settings.particles = 50000
settings.keff_trigger = {'type':'std_dev','threshold':0.00010}
settings.trigger_active = True
settings.trigger_max_batches = 50000
settings.output = {'tallies':True}

settings.temperature['method'] = 'interpolation'

model.settings = settings

### Tallies

In [17]:
tallies = openmc.Tallies()
model.tallies = tallies

### Define and choose energy group

In [18]:
egroup_name = 'SHEM-361'
egroup = openmc.mgxs.GROUP_STRUCTURES[egroup_name]

### Create MGXS tallies for mgxs directory

In [19]:
pn = 7
    
mgxs_lib_ = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(egroup)

mgxs_lib_.energy_groups = groups
mgxs_lib_.scatter_format = "legendre"
mgxs_lib_.mgxs_types = ['total', 'absorption', 'nu-fission', 'fission' ,'chi', \
                       'consistent nu-scatter matrix', 'multiplicity matrix','kappa-fission']

mgxs_lib_.legendre_order = pn
if pn == 0:
    mgxs_lib_.correction = None

mgxs_lib_.by_nuclide = False
mgxs_lib_.domain_type = 'cell'
mgxs_lib_.domains = model.geometry.get_all_material_cells().values()

all_cell_order=[]
for cell in (mgxs_lib_.domains):
    all_cell_order.append(cell.name)
# print(mgxs_lib_.domains)
mgxs_lib_.build_library()
print(all_cell_order)
    
mgxs_lib_.build_library()
mgxs_lib_.check_library_for_openmc_mgxs()

mgxs_lib_.add_to_tallies_file(tallies, merge=True)
    


['fuel_pincell', 'clad_pincell', 'moderator_pincell', 'gap_pincell', 'water_guide', 'clad_guide', 'moderator_guide', 'water_instru', 'clad_instru', 'moderator_instru', 'water_outside']


/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 7 is greater than zero
  warn(msg, RuntimeWarning)


### Create power tally

In [20]:
## model.export_to_model_xml('model_iso.xml')

### Run OpenMC model

In [21]:
statepoint_filename = model.run()

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=400.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=10.
  warn(msg, IDWarning)


                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

       56/1    1.16743    1.18257 +/- 0.00121
 Triggers unsatisfied, max unc./thresh. is 7.78856201898383 for eigenvalue
 The estimated number of batches is 2204
       57/1    1.17875    1.18247 +/- 0.00118
 Triggers unsatisfied, max unc./thresh. is 7.587579303018698 for eigenvalue
 The estimated number of batches is 2151
       58/1    1.18278    1.18247 +/- 0.00115
 Triggers unsatisfied, max unc./thresh. is 7.40036457370408 for eigenvalue
 The estimated number of batches is 2102
       59/1    1.17774    1.18235 +/- 0.00113
 Triggers unsatisfied, max unc./thresh. is 7.1646532941460945 for eigenvalue
 The estimated number of batches is 2022
       60/1    1.17750    1.18223 +/- 0.00110
 Triggers unsatisfied, max unc./thresh. is 7.051632293410104 for eigenvalue
 The estimated number of batches is 2010
       61/1    1.17715    1.18211 +/- 0.00108
 Triggers unsatisfied, max unc./thresh. is 6.866321786274776 for eigenvalue
 The estimated number of batches is 1954
       62/1    1.18013 

      107/1    1.18400    1.18229 +/- 0.00074
 Triggers unsatisfied, max unc./thresh. is 4.584265640097214 for eigenvalue
 The estimated number of batches is 1849
      108/1    1.18394    1.18231 +/- 0.00073
 Triggers unsatisfied, max unc./thresh. is 4.540079439619812 for eigenvalue
 The estimated number of batches is 1834
      109/1    1.18284    1.18232 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 4.4875020271755295 for eigenvalue
 The estimated number of batches is 1813
      110/1    1.17810    1.18227 +/- 0.00071
 Triggers unsatisfied, max unc./thresh. is 4.464748725733281 for eigenvalue
 The estimated number of batches is 1815
      111/1    1.18738    1.18233 +/- 0.00071
 Triggers unsatisfied, max unc./thresh. is 4.418428449040415 for eigenvalue
 The estimated number of batches is 1797
      112/1    1.18317    1.18234 +/- 0.00070
 Triggers unsatisfied, max unc./thresh. is 4.399409720363216 for eigenvalue
 The estimated number of batches is 1801
      113/1    1.1829

      158/1    1.17735    1.18208 +/- 0.00057
 Triggers unsatisfied, max unc./thresh. is 3.667882136497984 for eigenvalue
 The estimated number of batches is 1877
      159/1    1.18104    1.18208 +/- 0.00057
 Triggers unsatisfied, max unc./thresh. is 3.6408602198053783 for eigenvalue
 The estimated number of batches is 1863
      160/1    1.18227    1.18208 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 3.638801745132616 for eigenvalue
 The estimated number of batches is 1874
      161/1    1.18051    1.18207 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 3.6248261996471633 for eigenvalue
 The estimated number of batches is 1873
      162/1    1.17740    1.18203 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 3.652073505722379 for eigenvalue
 The estimated number of batches is 1914
      163/1    1.17560    1.18199 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 3.629417458410664 for eigenvalue
 The estimated number of batches is 1904
      164/1    1.185

...

     1433/1    1.17854    1.18162 +/- 0.00017
 Triggers unsatisfied, max unc./thresh. is 1.070542115263525 for eigenvalue
 The estimated number of batches is 1640
     1434/1    1.18649    1.18162 +/- 0.00017
 Triggers unsatisfied, max unc./thresh. is 1.069760279294839 for eigenvalue
 The estimated number of batches is 1639
     1435/1    1.18098    1.18162 +/- 0.00017
 Triggers unsatisfied, max unc./thresh. is 1.069069166470682 for eigenvalue
 The estimated number of batches is 1638
     1436/1    1.18971    1.18162 +/- 0.00017
 Triggers unsatisfied, max unc./thresh. is 1.0690151657698665 for eigenvalue
 The estimated number of batches is 1639
     1437/1    1.18175    1.18162 +/- 0.00017
 Triggers unsatisfied, max unc./thresh. is 1.0683203627978881 for eigenvalue
 The estimated number of batches is 1638
     1438/1    1.17336    1.18162 +/- 0.00017
 Triggers unsatisfied, max unc./thresh. is 1.0676075309615176 for eigenvalue
 The estimated number of batches is 1637
     1439/1    1.18

     1484/1    1.17891    1.18158 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0533677098306458 for eigenvalue
 The estimated number of batches is 1645
     1485/1    1.18124    1.18158 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0527188586038443 for eigenvalue
 The estimated number of batches is 1644
     1486/1    1.17485    1.18157 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0522606664794028 for eigenvalue
 The estimated number of batches is 1644
     1487/1    1.17579    1.18157 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0521654554576825 for eigenvalue
 The estimated number of batches is 1645
     1488/1    1.18846    1.18157 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0525414701782383 for eigenvalue
 The estimated number of batches is 1647
     1489/1    1.16636    1.18156 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0536088693601573 for eigenvalue
 The estimated number of batches is 1651
     1490/1    1

     1535/1    1.18316    1.18156 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0354606537718365 for eigenvalue
 The estimated number of batches is 1645
     1536/1    1.17813    1.18155 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0349030543244488 for eigenvalue
 The estimated number of batches is 1644
     1537/1    1.17979    1.18155 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.034237663667847 for eigenvalue
 The estimated number of batches is 1643
     1538/1    1.17871    1.18155 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.034106107177564 for eigenvalue
 The estimated number of batches is 1644
     1539/1    1.19232    1.18156 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.034802267090233 for eigenvalue
 The estimated number of batches is 1647
     1540/1    1.17347    1.18155 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0341714406086757 for eigenvalue
 The estimated number of batches is 1646
     1541/1    1.19

     1586/1    1.19299    1.18161 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0170147113536505 for eigenvalue
 The estimated number of batches is 1640
     1587/1    1.18200    1.18161 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0164002888413703 for eigenvalue
 The estimated number of batches is 1639
     1588/1    1.17333    1.18160 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0163525080544988 for eigenvalue
 The estimated number of batches is 1640
     1589/1    1.17201    1.18160 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0163591592136667 for eigenvalue
 The estimated number of batches is 1641
     1590/1    1.19385    1.18161 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0157396025627834 for eigenvalue
 The estimated number of batches is 1640
     1591/1    1.17461    1.18160 +/- 0.00016
 Triggers unsatisfied, max unc./thresh. is 1.0153930863979466 for eigenvalue
 The estimated number of batches is 1640
     1592/1    1


 =======================>     TIMING STATISTICS     <=======================

 Total time for initialization     = 3.5020e+00 seconds
   Reading cross sections          = 3.4659e+00 seconds
 Total time in simulation          = 1.1285e+03 seconds
   Time in transport only          = 1.1199e+03 seconds
   Time in inactive batches        = 8.3742e+00 seconds
   Time in active batches          = 1.1201e+03 seconds
   Time synchronizing fission bank = 1.1071e+00 seconds
     Sampling source sites         = 8.4232e-01 seconds
     SEND/RECV source sites        = 2.6396e-01 seconds
   Time accumulating tallies       = 6.2389e+00 seconds
   Time writing statepoints        = 5.1530e-01 seconds
 Total time for finalization       = 1.1527e+01 seconds
 Total time elapsed                = 1.1436e+03 seconds
 Calculation Rate (inactive)       = 119414 particles/second
 Calculation Rate (active)         = 72135.4 particles/second

 ============================>     RESULTS     <=====================

In [22]:
ce_spfile = f'./statepoint_{egroup_name}_ce.h5'
ce_sumfile = f'./summary_{egroup_name}_ce.h5'
print(ce_spfile)

./statepoint_SHEM-361_ce.h5


In [23]:
os.rename(statepoint_filename, ce_spfile)
os.rename('summary.h5', ce_sumfile)

In [24]:
# trick to make several runs work with jupyter
try:
    sp 
    print('sp found')
    sp.close()
except NameError:
    print('sp NOT found')

sp NOT found


In [25]:
sp = openmc.StatePoint(ce_spfile, autolink=False)
su = openmc.Summary(ce_sumfile)
sp.link_with_summary(su)

In [26]:
mgxs_lib_.load_from_statepoint(sp)
    
pb_name = '1A'
mgxs_lib_.create_mg_library(xs_type='macro', xsdata_names=all_cell_order).export_to_hdf5\
('mgxs_'+pb_name+'.h5')

## Clean up by deleting unwanted files

In [27]:
import os
import re

def delete_runtime_files(directory='.'):
    """
    Deletes all files with a .xml extension, files named statepoint.NNNNN.h5 where N is a digit,
    and the file named summary.h5 in the specified directory.

    Parameters:
    directory (str): The directory to search for files. Defaults to the current directory.
    """
    
    # Regular expression pattern to match files named statepoint.NNNNN.h5
    pattern = re.compile(r'statepoint\.\d{5}\.h5')

    # Iterate through files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # Check if the file has a .xml extension, matches the pattern, or is named summary.h5
        if filename.endswith('.xml') or pattern.match(filename) or filename == 'summary.h5' \
           or filename == 'tallies.out':
            # Delete the file
            try:
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")

# Example usage:
# delete_runtime_files('/path/to/directory')  # specify the directory path if needed

In [28]:
delete_runtime_files('./')

Deleted: ./plots.xml
Deleted: ./statepoint.00050.h5
Deleted: ./model.xml
Deleted: ./tallies.out
Deleted: ./materials.xml
Deleted: ./geometry.xml
